# Part 1 - Working with Text Data

### Use Python string methods remove irregular whitespace from the following string:

In [3]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "

print(whitespace_string)



  This is a    string   that has  
 a lot of  extra 
   whitespace.   


### Removal of irregular white spacing
One liner function

In [4]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "
print(" ".join(whitespace_string.split()))

This is a string that has a lot of extra whitespace.


### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://github.com/ryanleeallred/datasets/blob/master/dates.txt)

- Day
- Month
- Year


In [32]:
import re
import requests
import pandas as pd

In [120]:
# let's assign the raw data link to url
url = 'https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt'

In [121]:
# let's get the data
r = requests.get(url)

In [122]:
r.text

'March 8, 2015\r\nMarch 15, 2015\r\nMarch 22, 2015\r\nMarch 29, 2015\r\nApril 5, 2015\r\nApril 12, 2015\r\nApril 19, 2015\r\nApril 26, 2015\r\nMay 3, 2015\r\nMay 10, 2015\r\nMay 17, 2015\r\nMay 24, 2015\r\nMay 31, 2015\r\nJune 7, 2015\r\nJune 14, 2015\r\nJune 21, 2015\r\nJune 28, 2015\r\nJuly 5, 2015\r\nJuly 12, 2015\r\nJuly 19, 2015'

In [123]:
# Get month and Year because these are the easiest
Month = re.findall(r"[a-zA-z]+", r.text)
Year = re.findall(r"\d{4}", r.text)

In [124]:
# Get Day by getting all dates then storing only days into a vew var
day = re.findall(r"\d+", r.text)
Day = day[::2]

In [125]:
# Create the dataframe
df = pd.DataFrame({'Month':Month,
                  'Day': Day,
                  'Year':Year})

In [126]:
df.head()

,Month,Day,Year
0,March,8,2015
1,March,15,2015
2,March,22,2015
3,March,29,2015
4,April,5,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [137]:
# Let's get the data again
url = 'https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv'

In [138]:
# Read the data
df = pd.read_csv(url)

In [139]:
df.head()

,Sentiment,SentimentText
0,0,is so sad for my APL friend.............
1,0,I missed the New Moon trailer...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...
4,0,i think mi bf is cheating on me!!! T_T


In [140]:
df["SentimentText"].head()

0                         is so sad for my APL friend.............                                                                       
1                       I missed the New Moon trailer...                                                                                 
2                  omg its already 7:30 :O                                                                                               
3              .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...
4             i think mi bf is cheating on me!!!       T_T                                                                               
Name: SentimentText, dtype: object

In [141]:
df.isna().sum()

Sentiment        0
SentimentText    0
dtype: int64

In [142]:
# df['SentimentText'] = df['SentimentText'].apply(lambda x: re.sub('^b[\'|"]', '', x))
# df['SentimentText'] = df['SentimentText'].apply(lambda x: re.sub('.[\'|"]', '', x))
# df['SentimentText'] = df['SentimentText'].apply(lambda x: re.sub('<.*?>', '', x))

In [143]:
# Minimum cleaning to see how it looks
df['SentimentText'] = df['SentimentText'].apply(lambda x: re.sub(r'[^\w]', ' ', x))

In [144]:
df.head()

,Sentiment,SentimentText
0,0,is so sad for my APL friend
1,0,I missed the New Moon trailer
2,1,omg its already 7 30 O
3,0,Omgaga Im sooo im gunna CRy I ve been at this dentist since 11 I was suposed 2 just get a crown put on 30mins
4,0,i think mi bf is cheating on me T_T


In [145]:
# Required imports 
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /home/mishraka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mishraka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Let's merge all 4 required cleaning steps into one:

In [146]:
table = str.maketrans('', '', string.punctuation)
stop_words = set(stopwords.words('english'))

def nltk_tokenize(input):

    # Tokenize by word
    tokens = word_tokenize(input)
    #print("Tokens:", tokens)
    # Make all words lowercase
    lowercase_tokens = [w.lower() for w in tokens]
    #print("Lowercase:", lowercase_tokens)
    # Strip punctuation from within words
    no_punctuation = [x.translate(table) for x in lowercase_tokens]
    #print("No Punctuation:", no_punctuation)
    # Remove words that aren't alphabetic
    alphabetic = [word for word in no_punctuation if word.isalpha()]
    #print("Alphabetic:", alphabetic)
    # Remove stopwords
    words = [w for w in alphabetic if not w in stop_words]
    #print("Cleaned Words:", words)
    #print("--------------------------------")
    # Append to list
    return words
    

#### Tokenize df['SentimentText'] using the above defined function

In [147]:
# Create a new column for tokenized column df['SentimentText']
df['SentimentText_tokenized'] = df['SentimentText'].apply(nltk_tokenize)

In [148]:
df.head()

,Sentiment,SentimentText,SentimentText_tokenized
0,0,is so sad for my APL friend,"[sad, apl, friend]"
1,0,I missed the New Moon trailer,"[missed, new, moon, trailer]"
2,1,omg its already 7 30 O,"[omg, already]"
3,0,Omgaga Im sooo im gunna CRy I ve been at this dentist since 11 I was suposed 2 just get a crown put on 30mins,"[omgaga, im, sooo, im, gunna, cry, dentist, since, suposed, get, crown, put]"
4,0,i think mi bf is cheating on me T_T,"[think, mi, bf, cheating, tt]"


### How should TF-IDF scores be interpreted? How are they calculated?

### TF-IDF: 
 
Adopted from: https://towardsdatascience.com/machine-learning-text-processing-1d5a2d638958

1. Term Frequency (TF) = (Number of times term t appears in a document)/(Number of terms in the document)

2. Inverse Document Frequency (IDF) = log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in. The IDF of a rare word is high, whereas the IDF of a frequent word is likely to be low. Thus having the effect of highlighting words that are distinct.

3. We calculate TF-IDF value of a term as = TF * IDF

### My Understanding:
 
TF-IDF is a short term for term Frequency - Inverse Document frequency. It is an important algorithm that transforms text into meaningful numbers. TF-IDF is a widely used technique for feature extraction in NLP field.  

Term frequency: TF of a word is the frequency of a word in a document. For example: in a 200 word document, if a "hotel" appears 10 times then the TF for word "hotel" is 

TFhotel = 10/200 = 0.05

The IDF (Inverse Document Frequency) of a word is the measure of how important that term is in the whole corpus. For example: word 'hotel' appears x amount of times in a 10.000,000 million document-sized corpus (i.e. web). Now, let's assume that thjere are 0.3 millon documents that contain word 'hotel'. Then the IDF (i.e. log{DF}):

IDF(hotel) = log(10,000,000/300,000) = 1.52

therefore: Whotel = (TF * IDF)hotel = 0.05*1.52 = 0.076



# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.



In [149]:
sentiment_vector = []

for i in df['SentimentText_tokenized']:
    new_description = " ".join(i)
    sentiment_vector.append(new_description)
    
df['sentiment_vector'] = sentiment_vector

In [150]:
df.head()

,Sentiment,SentimentText,SentimentText_tokenized,sentiment_vector
0,0,is so sad for my APL friend,"[sad, apl, friend]",sad apl friend
1,0,I missed the New Moon trailer,"[missed, new, moon, trailer]",missed new moon trailer
2,1,omg its already 7 30 O,"[omg, already]",omg already
3,0,Omgaga Im sooo im gunna CRy I ve been at this dentist since 11 I was suposed 2 just get a crown put on 30mins,"[omgaga, im, sooo, im, gunna, cry, dentist, since, suposed, get, crown, put]",omgaga im sooo im gunna cry dentist since suposed get crown put
4,0,i think mi bf is cheating on me T_T,"[think, mi, bf, cheating, tt]",think mi bf cheating tt


In [97]:
df.isna().sum()

Sentiment                  0
SentimentText              0
SentimentText_tokenized    0
sentiment_vector           0
dtype: int64

### 1) Use Train_Test_Split to create train and test datasets.

In [151]:
# Train test split

from sklearn.model_selection import train_test_split

X = df.sentiment_vector
y = df.Sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [152]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(79991,)
(19998,)
(79991,)
(19998,)


### 2) Vectorize the tokenized documents using your choice of vectorization method.

### Count Vectorizer

In [153]:
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=400, ngram_range=(1,1), stop_words='english')
vectorizer.fit(X_train)
print(vectorizer.vocabulary_)

{'sleep': 287, 'watching': 366, 'quot': 255, 'school': 276, 'haha': 126, 'forgot': 98, 'la': 170, 'love': 192, 'music': 221, 'little': 181, 'got': 119, 'ta': 312, 'dont': 72, 'know': 169, 'coffee': 51, 'lt': 195, 'gt': 121, 'twitpic': 348, 'working': 381, 'wan': 361, 'na': 222, 'hear': 137, 'bet': 27, 'cute': 61, 'like': 177, 'friend': 102, 'told': 334, 'youtube': 399, 'video': 358, 'week': 369, 'ago': 3, 'http': 154, 'com': 53, 'time': 329, 'long': 185, 'party': 237, 'kinda': 167, 'thing': 320, 'good': 118, 'night': 227, 'sun': 306, 'way': 367, 'today': 333, 'yes': 397, 'sigh': 285, 'cool': 58, 'things': 321, 'hot': 150, 'day': 65, 'right': 266, 'lol': 184, 'new': 224, 'nice': 226, 'meet': 209, 'happy': 133, 'sad': 270, 'come': 54, 'makes': 202, 'mad': 200, 'yeah': 393, 'remember': 263, 'xxx': 389, 'yep': 396, 'think': 322, 'old': 232, 'far': 85, 'im': 159, 'car': 45, 'iphone': 161, 'really': 261, 'want': 362, 'bad': 22, 'ones': 234, 'miss': 212, 'favorite': 86, 'person': 240, 'help':

In [154]:
train_word_counts =vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(79991, 400)


,able,actually,add,ago,agree,ah,album,alexalltimelow,amazing,amp,andyclemmensen,anymore,aplusk,ashleytisdale,ask,ass,aw,away,awesome,aww,awww,baby,bad,beautiful,bed,believe,best,bet,better,big,birthday,bit,blog,boo,book,bored,bout,boy,break,bring,btw,busy,buy,called,came,car,care,cause,chance,change,check,coffee,cold,com,come,coming,computer,congrats,cool,course,crazy,cute,cuz,dad,damn,day,days,dear,definitely,didnt,dm,dog,dont,dude,early,eat,email,end,enjoy,excited,face,facebook,fail,family,fan,far,favorite,feel,feeling,ff,finally,fine,follow,followers,followfriday,following,food,forget,forgot,forward,free,friday,friend,friends,fuck,fun,funny,game,gets,getting,girl,girls,glad,god,goes,going,gon,gone,good,got,great,gt,guess,guy,guys,ha,haha,hahaha,hair,half,hang,happen,happened,happy,hard,hate,head,hear,heard,heart,hehe,hell,hello,help,hey,hi,hit,home,hope,hopefully,hot,hour,hours,house,http,hugs,hurt,idea,ill,im,internet,iphone,jealous,job,june,kids,kind,kinda,knew,know,la,late,later,leave,left,let,life,like,link,list,listen,little,live,lmao,lol,long,look,looking,looks,lost,lot,lots,love,loved,lovely,lt,luck,lucky,lunch,ly,mad,make,makes,making,man,maybe,mean,means,meant,meet,message,mind,miss,missed,missing,mom,monday,money,month,morning,movie,music,na,need,new,news,nice,night,nope,oh,ok,okay,old,omg,ones,online,open,party,pay,people,person,phone,pic,pics,picture,place,play,playing,poor,post,ppl,pretty,probably,problem,quite,quot,rain,read,reading,ready,real,really,reason,remember,reply,rest,right,rock,room,run,sad,said,saturday,saw,say,says,school,season,seeing,seen,send,sent,seriously,shit,sick,sigh,site,sleep,song,soo,soon,sooo,sorry,sound,sounds,start,started,stay,stop,story,stuck,stuff,stupid,suck,sucks,summer,sun,sunday,super,supposed,sure,sweet,ta,taking,talk,talking,tell,thank,thanks,thats,thing,things,think,thinking,tho,thought,thx,til,till,time,times,tinyurl,tired,today,told,tomorrow,tonight,took,totally,train,tried,trip,true,try,trying,tv,tweet,tweets,twitpic,twitter,ugh,understand,unfortunately,update,ur,use,used,using,video,wait,waiting,wan,want,wanted,watch,watched,watching,way,weather,week,weekend,weeks,welcome,went,win,wish,wonderful,wont,word,words,work,working,works,world,wow,wrong,www,xd,xx,xxx,ya,yay,yea,yeah,year,years,yep,yes,yesterday,youtube
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [106]:
X_test.head()

33965    allieandra wheeee                                                                                     
22853    thank ff good meet ya                                                                                 
19448    electricbath eewwww gross sorry hayward hates like                                                    
9732     followfriday little late special shoutout somersetmarcy missus                                        
7129     icanhelp shopping deals personal assistant event planning get together services jennifer git gmail com
Name: sentiment_vector, dtype: object

In [107]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), 
                                 columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(19998, 400)


,able,actually,add,ago,agree,ah,alexalltimelow,amazing,amp,andyclemmensen,anymore,aplusk,aret,ashleytisdale,ask,aw,away,awesome,aww,awww,baby,bad,beautiful,bed,believe,best,bet,better,big,birthday,bit,blog,boo,book,bored,boy,break,bring,btw,busy,buy,called,came,car,care,cat,cause,chance,change,check,coffee,cold,com,come,coming,congrats,cool,couldt,course,crazy,cute,cuz,damn,day,days,dear,definitely,didnt,didt,dm,doest,dont,dot,dude,early,eat,email,end,enjoy,excited,face,facebook,fail,family,fan,far,favorite,feel,feeling,ff,finally,fine,follow,followers,followfriday,following,food,forgot,forward,free,friday,friend,friends,fun,funny,game,gets,getting,girl,girls,glad,god,goes,going,gon,gone,good,got,great,gt,guess,guy,guys,ha,haha,hahaha,hair,hang,happen,happened,happy,hard,hate,havet,head,hear,heard,heart,hehe,hell,hello,help,hey,hi,hit,home,hope,hopefully,hot,hour,hours,house,hs,http,hugs,hurt,idea,ill,im,internet,iphone,ist,jealous,job,kids,kind,kinda,knew,know,late,later,leave,left,les,let,life,like,link,list,listen,little,live,lmao,lol,long,look,looking,looks,lost,lot,lots,love,loved,lovely,lt,luck,lucky,lunch,ly,mad,make,makes,making,man,maybe,mean,means,meet,mind,miss,missed,missing,mom,monday,money,month,morning,movie,music,na,need,new,news,nice,night,nope,oh,ok,okay,old,omg,ones,online,party,pay,people,person,phone,pic,pics,picture,place,play,playing,poor,post,ppl,pretty,probably,problem,quite,quot,rain,read,reading,ready,real,really,reason,remember,reply,rest,right,rock,room,run,sad,said,saturday,saw,say,says,school,seeing,seen,send,sent,seriously,shit,shs,sick,sigh,site,sleep,song,soo,soon,sooo,sorry,sound,sounds,start,started,stay,stop,story,stuff,stupid,suck,sucks,summer,sun,sunday,super,sure,sweet,ta,taking,talk,talking,tell,thank,thanks,thas,thats,thers,thing,things,think,thinking,tho,thought,thx,till,time,times,tinyurl,tired,today,told,tomorrow,tonight,took,totally,train,tried,trip,true,try,trying,tv,tweet,tweets,twitpic,twitter,ugh,unfortunately,ur,use,used,using,video,wait,waiting,wan,want,wanted,wast,watch,watched,watching,way,weather,week,weekend,weeks,welcome,went,whas,win,wish,wll,wonderful,wont,word,words,work,working,works,world,wot,wouldt,wow,wre,wrong,www,xd,xx,ya,yay,yea,yeah,year,years,yep,yes,yesterday,yoll,yore,yove
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### TF-IDF Vectorization Method

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=400, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)
print(vectorizer.vocabulary_)

{'cat': 45, 'sleep': 283, 'watching': 360, 'quot': 251, 'school': 272, 'haha': 124, 'forgot': 97, 'love': 191, 'music': 218, 'little': 180, 'got': 117, 'ta': 306, 'dont': 71, 'know': 168, 'coffee': 50, 'lt': 194, 'gt': 119, 'twitpic': 343, 'working': 377, 'wan': 354, 'na': 219, 'hear': 135, 'bet': 26, 'cute': 60, 'like': 176, 'friend': 101, 'told': 329, 'video': 351, 'week': 363, 'ago': 3, 'http': 153, 'com': 52, 'time': 324, 'long': 184, 'party': 233, 'kinda': 166, 'thing': 316, 'good': 116, 'night': 224, 'yore': 398, 'sun': 301, 'way': 361, 'thas': 313, 'today': 328, 'yes': 395, 'sigh': 281, 'cool': 56, 'things': 317, 'hot': 148, 'day': 63, 'right': 262, 'lol': 183, 'new': 221, 'nice': 223, 'meet': 207, 'happy': 130, 'sad': 266, 'come': 53, 'makes': 201, 'mad': 199, 'yeah': 391, 'remember': 259, 'yep': 394, 'doest': 70, 'think': 318, 'old': 229, 'far': 85, 'im': 158, 'whas': 368, 'thers': 315, 'car': 43, 'iphone': 160, 'really': 257, 'want': 355, 'dot': 72, 'bad': 21, 'ones': 231, 'm

### Vectorize Training Data

In [110]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(79991, 400)


,able,actually,add,ago,agree,ah,alexalltimelow,amazing,amp,andyclemmensen,anymore,aplusk,aret,ashleytisdale,ask,aw,away,awesome,aww,awww,baby,bad,beautiful,bed,believe,best,bet,better,big,birthday,bit,blog,boo,book,bored,boy,break,bring,btw,busy,buy,called,came,car,care,cat,cause,chance,change,check,coffee,cold,com,come,coming,congrats,cool,couldt,course,crazy,cute,cuz,damn,day,days,dear,definitely,didnt,didt,dm,doest,dont,dot,dude,early,eat,email,end,enjoy,excited,face,facebook,fail,family,fan,far,favorite,feel,feeling,ff,finally,fine,follow,followers,followfriday,following,food,forgot,forward,free,friday,friend,friends,fun,funny,game,gets,getting,girl,girls,glad,god,goes,going,gon,gone,good,got,great,gt,guess,guy,guys,ha,haha,hahaha,hair,hang,happen,happened,happy,hard,hate,havet,head,hear,heard,heart,hehe,hell,hello,help,hey,hi,hit,home,hope,hopefully,hot,hour,hours,house,hs,http,hugs,hurt,idea,ill,im,internet,iphone,ist,jealous,job,kids,kind,kinda,knew,know,late,later,leave,left,les,let,life,like,link,list,listen,little,live,lmao,lol,long,look,looking,looks,lost,lot,lots,love,loved,lovely,lt,luck,lucky,lunch,ly,mad,make,makes,making,man,maybe,mean,means,meet,mind,miss,missed,missing,mom,monday,money,month,morning,movie,music,na,need,new,news,nice,night,nope,oh,ok,okay,old,omg,ones,online,party,pay,people,person,phone,pic,pics,picture,place,play,playing,poor,post,ppl,pretty,probably,problem,quite,quot,rain,read,reading,ready,real,really,reason,remember,reply,rest,right,rock,room,run,sad,said,saturday,saw,say,says,school,seeing,seen,send,sent,seriously,shit,shs,sick,sigh,site,sleep,song,soo,soon,sooo,sorry,sound,sounds,start,started,stay,stop,story,stuff,stupid,suck,sucks,summer,sun,sunday,super,sure,sweet,ta,taking,talk,talking,tell,thank,thanks,thas,thats,thers,thing,things,think,thinking,tho,thought,thx,till,time,times,tinyurl,tired,today,told,tomorrow,tonight,took,totally,train,tried,trip,true,try,trying,tv,tweet,tweets,twitpic,twitter,ugh,unfortunately,ur,use,used,using,video,wait,waiting,wan,want,wanted,wast,watch,watched,watching,way,weather,week,weekend,weeks,welcome,went,whas,win,wish,wll,wonderful,wont,word,words,work,working,works,world,wot,wouldt,wow,wre,wrong,www,xd,xx,ya,yay,yea,yeah,year,years,yep,yes,yesterday,yoll,yore,yove
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.364729,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.453129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.486653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0

### Vecotrize Test Data

In [111]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(19998, 400)


,able,actually,add,ago,agree,ah,alexalltimelow,amazing,amp,andyclemmensen,anymore,aplusk,aret,ashleytisdale,ask,aw,away,awesome,aww,awww,baby,bad,beautiful,bed,believe,best,bet,better,big,birthday,bit,blog,boo,book,bored,boy,break,bring,btw,busy,buy,called,came,car,care,cat,cause,chance,change,check,coffee,cold,com,come,coming,congrats,cool,couldt,course,crazy,cute,cuz,damn,day,days,dear,definitely,didnt,didt,dm,doest,dont,dot,dude,early,eat,email,end,enjoy,excited,face,facebook,fail,family,fan,far,favorite,feel,feeling,ff,finally,fine,follow,followers,followfriday,following,food,forgot,forward,free,friday,friend,friends,fun,funny,game,gets,getting,girl,girls,glad,god,goes,going,gon,gone,good,got,great,gt,guess,guy,guys,ha,haha,hahaha,hair,hang,happen,happened,happy,hard,hate,havet,head,hear,heard,heart,hehe,hell,hello,help,hey,hi,hit,home,hope,hopefully,hot,hour,hours,house,hs,http,hugs,hurt,idea,ill,im,internet,iphone,ist,jealous,job,kids,kind,kinda,knew,know,late,later,leave,left,les,let,life,like,link,list,listen,little,live,lmao,lol,long,look,looking,looks,lost,lot,lots,love,loved,lovely,lt,luck,lucky,lunch,ly,mad,make,makes,making,man,maybe,mean,means,meet,mind,miss,missed,missing,mom,monday,money,month,morning,movie,music,na,need,new,news,nice,night,nope,oh,ok,okay,old,omg,ones,online,party,pay,people,person,phone,pic,pics,picture,place,play,playing,poor,post,ppl,pretty,probably,problem,quite,quot,rain,read,reading,ready,real,really,reason,remember,reply,rest,right,rock,room,run,sad,said,saturday,saw,say,says,school,seeing,seen,send,sent,seriously,shit,shs,sick,sigh,site,sleep,song,soo,soon,sooo,sorry,sound,sounds,start,started,stay,stop,story,stuff,stupid,suck,sucks,summer,sun,sunday,super,sure,sweet,ta,taking,talk,talking,tell,thank,thanks,thas,thats,thers,thing,things,think,thinking,tho,thought,thx,till,time,times,tinyurl,tired,today,told,tomorrow,tonight,took,totally,train,tried,trip,true,try,trying,tv,tweet,tweets,twitpic,twitter,ugh,unfortunately,ur,use,used,using,video,wait,waiting,wan,want,wanted,wast,watch,watched,watching,way,weather,week,weekend,weeks,welcome,went,whas,win,wish,wll,wonderful,wont,word,words,work,working,works,world,wot,wouldt,wow,wre,wrong,www,xd,xx,ya,yay,yea,yeah,year,years,yep,yes,yesterday,yoll,yore,yove
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### Logistic Regression

In [114]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

/home/mishraka/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train Accuracy: 0.714967933892563
Test Accuracy: 0.711021102110211


In [115]:
from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.7092422897575977
Test Accuracy: 0.7063206320632063


In [116]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

/home/mishraka/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train Accuracy: 0.8822617544473753
Test Accuracy: 0.6874187418741874


# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [117]:
df.head()

,Sentiment,SentimentText,SentimentText_tokenized,sentiment_vector
0,0,is so sad for my APL friend,"[sad, apl, friend]",sad apl friend
1,0,I missed the New Moon trailer,"[missed, new, moon, trailer]",missed new moon trailer
2,1,omg its already 7 30 O,"[omg, already]",omg already
3,0,Omgaga Im sooo im gunna CRy ve been at this dentist since 11 I was suposed 2 just get a crown put on 30mins,"[omgaga, im, sooo, im, gunna, cry, dentist, since, suposed, get, crown, put]",omgaga im sooo im gunna cry dentist since suposed get crown put
4,0,i think mi bf is cheating on me T_T,"[think, mi, bf, cheating, tt]",think mi bf cheating tt


### 1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset.

In [155]:
import gensim
from gensim.models import Word2Vec
w2v = Word2Vec(df['SentimentText_tokenized'], min_count=20, window=3, size=500, negative=20)

In [163]:
words = list(w2v.wv.vocab)
print(f'Vocablury Size: {len(words)}')

Vocablury Size: 3689


### 2) Display the 10 words that are most similar to the word "twitter"

In [156]:
w2v.wv.most_similar('twitter', topn=10)

[('dm', 0.7838777303695679),
 ('info', 0.7752491235733032),
 ('email', 0.7659730911254883),
 ('link', 0.7581459283828735),
 ('updates', 0.7571792006492615),
 ('facebook', 0.7568240165710449),
 ('via', 0.7420700192451477),
 ('account', 0.7407490611076355),
 ('direct', 0.7391822934150696),
 ('page', 0.7353038787841797)]

In [157]:
w2v.wv.similarity("twitter", 'itunes')

0.64450204

In [158]:
w2v.wv.similarity("twitter", 'lunch')

0.0069465544

In [159]:
w2v.wv.doesnt_match(["twitter", 'lunch', 'email'])

'lunch'

In [160]:
### Let's change the hyperparameters a little bit

In [135]:
import gensim
from gensim.models import Word2Vec
w2v = Word2Vec(df['SentimentText_tokenized'], min_count=20, window=5, size=200, negative=20)

In [162]:
w2v.wv.most_similar('link', topn=15)

[('facebook', 0.9170545935630798),
 ('website', 0.9131760001182556),
 ('site', 0.9088238477706909),
 ('iamsoannoyed', 0.9064961075782776),
 ('youtube', 0.9057779312133789),
 ('email', 0.9042657613754272),
 ('blog', 0.9020353555679321),
 ('info', 0.8974170684814453),
 ('myspace', 0.8930231928825378),
 ('flickr', 0.8906173706054688),
 ('dm', 0.8753321170806885),
 ('check', 0.8737671971321106),
 ('page', 0.8705669641494751),
 ('via', 0.8681374788284302),
 ('address', 0.8633331656455994)]